## Init methods

In [1]:
from pydantic import BaseModel, BaseSettings, root_validator, validator

class Config(BaseSettings):
    option1: str
    option2: int


class Evaluator(BaseModel):

    config: Config

    @root_validator(pre=True)
    def validate_config(cls, values):

        # check if config is instance of Config
        if values.get("config"):
            return values

        else: 
            return {"config": Config(**values)}


    

In [2]:
evaluator = Evaluator(option1="hi", option2=3)
evaluator

Evaluator(config=Config(option1='hi', option2=3))

In [3]:
config = Config(option1="hi", option2=3)
config

Config(option1='hi', option2=3)

In [4]:
evaluator_from_config = Evaluator(config=config)
evaluator_from_config

Evaluator(config=Config(option1='hi', option2=3))

# JSON encoding and decoding callable functions

In [5]:
import json
from typing import Callable
from types import FunctionType, MethodType

from pydantic import Extra

from xopt.utils import get_function 

In [6]:
JSON_ENCODERS = {
    FunctionType: lambda x: f"{x.__module__}.{x.__qualname__}",
    Callable: lambda x: f"{x.__module__}.{type(x).__qualname__}",
}

from typing import Any, Callable, Dict, Generic, Iterable, Optional, TypeVar, Tuple
ObjType = TypeVar("ObjType")
JSON_ENCODERS = {
    # function/method type distinguished for class members and not recognized as callables
    FunctionType: lambda x: f"{x.__module__}.{x.__qualname__}",
    MethodType: lambda x: f"{x.__module__}.{x.__qualname__}",
    Callable: lambda x: f"{x.__module__}.{type(x).__qualname__}",
    type: lambda x: f"{x.__module__}.{x.__name__}",
    # for encoding instances of the ObjType}
    ObjType: lambda x: f"{x.__module__}.{x.__class__.__qualname__}",
}


In [7]:

class CallableModel(BaseModel):
    callable: Callable

    class Config:
        arbitrary_types_allowed = True
        json_encoders = JSON_ENCODERS
        extra = 'forbid'

    @root_validator(pre=True)
    def validate_all(cls, values):

        callable = values.pop("callable")
        if not isinstance(
            callable,
            (
                str,
                Callable,
            ),
        ):
            raise ValueError(
                "Callable must be object or a string. Provided %s", type(callable)
            )

        values["callable"] = get_function(callable)



        return values

In [8]:
def f(x):
    return 2*x

In [9]:
m = CallableModel(callable=f)
m.callable(3)

6

In [10]:
m.json()

'{"callable": "__main__.f"}'

In [11]:
m2 = CallableModel(**json.loads(m.json()))
m2.callable(345)

690

In [12]:
def f():
    pass

# or this
f = lambda x: 2*x

type(f) is FunctionType

True

# Evaluator

In [13]:
import json
from typing import Callable
from types import FunctionType, MethodType

from pydantic import Extra, Field

from xopt.pydantic import NormalExecutor
from xopt.utils import get_function, get_function_defaults

In [14]:
from concurrent.futures import ProcessPoolExecutor
from xopt.evaluator import DummyExecutor

JSON_ENCODERS = {
    FunctionType: lambda x: f"{x.__module__}.{x.__qualname__}",
    Callable: lambda x: f"{x.__module__}.{type(x).__qualname__}",
}

class Evaluator(BaseModel):
    function: Callable
    max_workers: int = 1
    executor: NormalExecutor = Field(exclude=True)
    function_kwargs: dict = {}
    

    class Config:
        arbitrary_types_allowed = True
        # validate_assignment = True # Broken in 1.9.0. Trying to fix in https://github.com/samuelcolvin/pydantic/pull/4194
        json_encoders = JSON_ENCODERS
        extra = 'forbid'

    @root_validator(pre=True)
    def validate_all(cls, values):
   
        f = get_function(values["function"])
        kwargs = values.get("function_kwargs", {})
        kwargs = {**get_function_defaults(f), **kwargs}
        values["function"] = f
        values["function_kwargs"] = kwargs

        max_workers = values.pop("max_workers", 1)

        executor = values.pop("executor", None)
        if not executor:
            if max_workers > 1:
                executor = ProcessPoolExecutor(max_workers=max_workers)
            else: 
                executor = DummyExecutor()

        # Cast as a NormalExecutor
        values["executor"] =  NormalExecutor[type(executor)](executor=executor)
        values["max_workers"] = max_workers
        
        return values    



def g(a, b=2):
    return a*b

ev = Evaluator(function=g, function_kwargs={'b':3}, max_workers=2, executor=None)
ev.executor = None
ev.json()

'{"function": "__main__.g", "max_workers": 2, "function_kwargs": {"b": 3}}'

In [15]:
ev.executor 

In [16]:
ev.max_workers = 1
ev.executor=None

In [17]:
ev.json()

'{"function": "__main__.g", "max_workers": 1, "function_kwargs": {"b": 3}}'

# Executors

In [18]:
from concurrent.futures import ThreadPoolExecutor

In [19]:
with ThreadPoolExecutor() as executor:
    #print(dir(executor))
    print(type(executor))

<class 'concurrent.futures.thread.ThreadPoolExecutor'>


In [20]:
from xopt.pydantic import NormalExecutor

In [21]:
type(executor)

concurrent.futures.thread.ThreadPoolExecutor

In [22]:
NormalExecutor[type(executor)](executor=executor)

NormalExecutor[ThreadPoolExecutor](loader=ObjLoader[ThreadPoolExecutor](object=None, loader=CallableModel(callable=<class 'concurrent.futures.thread.ThreadPoolExecutor'>, kwargs=Kwargs_ThreadPoolExecutor(max_workers=None, initializer=None, initargs=None, thread_name_prefix='')), object_type=<class 'concurrent.futures.thread.ThreadPoolExecutor'>), executor_type=<class 'concurrent.futures.thread.ThreadPoolExecutor'>, submit_callable='submit', map_callable='map', shutdown_callable='shutdown', executor=<concurrent.futures.thread.ThreadPoolExecutor object at 0x147e92670>)